In [1]:
import os.path
import numpy as np
import pandas as pd
from time import time
from os import listdir
from os.path import isfile, join

In [2]:
def file_csv_list(target_path):
    return [f for f in listdir(target_path) if isfile(join(target_path, f))]

In [3]:
def read_csv(filename):
    # read revise csv file and print cost time
    t0 = time()
    df = pd.read_csv(filename, error_bad_lines=False)
    print("time for read csv: %.2f" % (time()-t0))
    return df

In [4]:
def remove_date_column(df):
    # we don't use VOICE_DATE/VOICE_MONTH bcz it is a monthly data
    if("VOICE_DATE" in df.columns):
        df = df.drop('VOICE_DATE', 1)
        print("Drop VOICE_DATE")
    if("VOICE_MONTH" in df.columns):
        df = df.drop('VOICE_MONTH', 1)
        print("Drop VOICE_MONTH")
    return df

In [5]:
def save_dataframe(df, out_filename):
   # write to csv and no index
    t0 = time()
    df.to_csv(out_filename + ".csv", index=False, encoding='utf-8')
    print("time for output csv file: %.2f" % (time()-t0))

In [6]:
def split_work_holiday(df):
    # let it sort mon ~ sun
    cnt_col_list = df.columns[2:8]
    cnt_col_list = cnt_col_list.append(df.columns[1:2])
    time_col_list = df.columns[9:15]
    time_col_list = time_col_list.append(df.columns[8:9])
    prefix = "VOICE_MT_"
    if "VOICE_MO_HOUR_SUN_CNT" in df.columns:
        prefix = "VOICE_MO_"
 
    # sum
    df[prefix + "workday_cnt"] = 0
    for col in cnt_col_list[:5]:
        df[prefix + "workday_cnt"] += df[col]
    # sum
    df[prefix + "workday_cnt_avg"] = 0
    df[prefix + "workday_cnt_avg"] =df[prefix + "workday_cnt"] / 5
            
    df[prefix + "workday_time"] = 0
    for col in time_col_list[:5]:
        df[prefix + "workday_time"] += df[col]
    df[prefix + "workday_time_avg"] = 0
    df[prefix + "workday_time_avg"] =df[prefix + "workday_time"] / 5
    
    df[prefix + "holiday_cnt"] = 0
    for col in cnt_col_list[5:]:
        df[prefix + "holiday_cnt"] += df[col]
    df[prefix + "holiday_cnt_avg"] = 0
    df[prefix + "holiday_cnt_avg"] =df[prefix + "holiday_cnt"] / 2
    
    df[prefix + "holiday_time"] = 0
    for col in time_col_list[5:]:
        df[prefix + "holiday_time"] += df[col]
    df[prefix + "holiday_time_avg"] = 0
    df[prefix + "holiday_time_avg"] =df[prefix + "holiday_time"] / 2
    return df

In [ ]:
# set configure
# path = "../DATA_FULL/"
path = "./CDR_MONTHLY_AGGR/"
# filename = "CDR_MO_HOUR_revise_month_1"
# relative_filename = path + filename + ".csv"
# out_path = "./CDR_MONTHLY_WORKDAY/"
out_path = "./CDR_MONTHLY_AGGR/"

for filename in file_csv_list(path):
    if ".csv" in filename and "WEEK" in filename:
        filename_none_postfix = filename[:-4]
        out_filename =  out_path + filename_none_postfix + "_workday"
        df = read_csv(path + filename)
        df = split_work_holiday(df)
        save_dataframe(df, out_filename)
#         print(df.columns)

time for read csv: 4.79
time for output csv file: 53.20
time for read csv: 4.78


In [ ]:
df.head()